In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import torch
from tqdm import tqdm
from kornia.geometry import warp_perspective


path_to_files = Path('rawData/calib')
dynamic, static = {}, {}
for i, path in enumerate(path_to_files.glob('left_*.npy')):
    idx = int(path.stem.split('left_')[-1])
    if not (path.parent / f"right_{idx}.npy").exists():
        print(f"Skipping {idx}")
        continue
    dynamic[idx] = np.load(path_to_files / f'left_{idx}.npy')
    static[idx] = np.load(path_to_files / f'right_{idx}.npy')
dynamic = np.array([dynamic[i] for i in sorted(dynamic.keys())])
static = np.array([static[i] for i in sorted(static.keys())])
dynamic, static = np.array(dynamic), np.array(static)

print(f"Loaded {len(dynamic)} frames")

# Build and save the mean homography matrix

In [ ]:
h = np.eye(3, dtype=float)

h[2, 2] = 1
h[2, 1] = -1e-6
h[2, 0] = 1e-6

h[1, 2] = -0.78
h[1, 1] = 0.999
h[1, 0] = -0.015

h[0, 2] = -5.2
h[0, 1] = 0.017
h[0, 0] = 0.9998

h

# Examples of results with the mean homography

In [ ]:
for idx, (dyn, stat) in enumerate(zip(dynamic, static)):
    warped = cv2.warpPerspective(dyn, h, list(reversed(dyn.shape[-2:])))
    mask = cv2.warpPerspective(
        np.ones_like(dyn),
        h, list(reversed(stat.shape[-2:])),
        flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=0).astype(bool)

    # Erode mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.erode(mask.astype(np.uint8), kernel, iterations=1).astype(bool)

    vmin = warped[mask].min()
    vmax = warped[mask].max()

    # Plot the dynamic, static and warped images
    plt.figure(figsize=(20, 5))
    plt.subplot(1, 4, 1)
    plt.imshow(dyn)
    plt.title('Dynamic')
    plt.subplot(1, 4, 2)
    plt.imshow(stat)
    plt.title('Static')
    plt.subplot(1, 4, 3)
    plt.imshow(warped, vmin=vmin, vmax=vmax)
    plt.title('Warped')
    ax = plt.subplot(1, 4, 4)
    diff = np.abs(warped.astype(float) - stat.astype(float))
    diff[~mask] = np.nan

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(ax.imshow(diff, cmap='bwr'), cax=cax)
    ax.set_title(f'Diff, avg err: {diff[mask].mean():.2g}GL')

    plt.title('Difference')
    plt.show()
    plt.close()

    # if idx > 20:
    #     break

# Optimize the homography matrix with SGD

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE = torch.float32

dyn = torch.from_numpy(dynamic[:, None].astype(float)).to(DEVICE, DTYPE)
stat = torch.from_numpy(static[:, None].astype(float)).to(DEVICE, DTYPE)
h_tensor = torch.from_numpy(h)[None,  ...].to(DEVICE, DTYPE)
h_tensor.requires_grad_(True)
h_orig = h_tensor.clone()

optim = torch.optim.AdamW([h_tensor], lr=1e-6)
loss_fn = torch.nn.L1Loss()

N_EPOCHS = 4000
with tqdm(total=N_EPOCHS) as pbar:
    for i in range(N_EPOCHS):
        warped = warp_perspective(dyn, h_tensor, dsize=stat.shape[-2:], mode='bicubic', align_corners=True)
        loss = loss_fn(warped, stat)
        pbar.set_postfix_str(f'Loss: {loss.item():.2g}')
        optim.zero_grad()
        loss.backward()
        optim.step()
        pbar.update()

print(f"Original homography:\n{h_orig.detach().cpu().numpy()[0]}", flush=True)
print(f"Optimized homography:\n{h_tensor.detach().cpu().numpy()[0]}", flush=True)

In [ ]:
print(f"Original homography:\n{h_orig.detach().cpu().numpy()[0]}", flush=True)
print(f"Optimized homography:\n{h_tensor.detach().cpu().numpy()[0]}", flush=True)

In [ ]:
h_new = h_tensor.detach().cpu().numpy()[0]
for idx, path in enumerate(path_to_files.glob('M_*.npy')):
    idx_of_frame = int(path.stem.split('_')[-1])
    warped = cv2.warpPerspective(dynamic[idx], h, list(reversed(static.shape[-2:])))
    mask = warped != 0

    # Erode mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.erode(mask.astype(np.uint8), kernel, iterations=1).astype(bool)

    vmin = warped[mask].min()
    vmax = warped[mask].max()

    # Plot the dynamic, static and warped images
    plt.figure(figsize=(20, 5))
    plt.subplot(1, 4, 1)
    plt.imshow(dynamic[idx])
    plt.title('Dynamic')
    plt.subplot(1, 4, 2)
    plt.imshow(static[idx])
    plt.title('Static')
    plt.subplot(1, 4, 3)
    plt.imshow(warped, vmin=vmin, vmax=vmax)
    plt.title('Warped')
    ax = plt.subplot(1, 4, 4)
    diff = np.abs(warped.astype(float) - static[idx].astype(float))
    diff[~mask] = np.nan

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(ax.imshow(diff, cmap='bwr'), cax=cax)
    ax.set_title(f'Diff, avg err: {diff[mask].mean():.2g}GL')

    plt.title('Difference')
    plt.show()
    plt.close()

    if idx > 3:
        break